<a href="https://colab.research.google.com/github/RodrigoGalvan/Actividad1_Aspiradora/blob/main/Actividad1_Aspiradora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 1

Integrantes de equipo:

*   Rodrigo Galván Paiz
*   Andres Aguirre Rodriguez
*   Sergio Alejandro Esparza González

La siguiente actividad consistio de tener un recuadro con cuadros en los cuales se generaran el color negro o be3ige, los negros siendo cuadros sucios y los beige siendo limpios. Adicionalmente a estos en la fila superior del recuadro se encuentran 12 aspiradoras que de manera aleatoria se mueven hasta encontrar cuadros con basura y asismismo limpiarlos. Junto con todo esto tambien se tiene un sistema que cuenta cuantos espacios se movio cada aspiradora, el tiempo transcurrido, y el porcentaje de cuadros limpios que se encuentran al terminar la simulacion.


In [ ]:
pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import MultiGrid
from mesa.space import SingleGrid


# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/latest/tutorials/intro_tutorial.html). Se modificó para que funcionara con el presente problema pero en esencia es lo mismo.

In [ ]:
# Clase que define las caracteristicas de las aspiradoras
class GameLifeVacuum(Agent):
    
    # Inicializacion de valores de las aspiradoras
    def __init__(self, unique_id, model):     
        super().__init__(unique_id, model)
        self.live = np.random.choice([1]) 
        self.charge = 100 #Empieza con 100 de carga 
        self.one = False
        self.count = 0
    
    # Definicion de la funcion que tiene las acciones que hace cada aspiradora cada iteracion
    def step(self):        

        #Definir los vecinos de el recuadro donde se encuentra la aspiradora
        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)

        # Si la aspiradora se encuentra en una de las casillas de la primera fila, adquiere una carga de 40
        if self.pos[0] == 0:
            self.charge = self.charge + 40
          
        # Si la aspiradora encuentra basura en una de las casillas vecinas, se mueve hacia ella y se le resta 20 de carga
        for neighbor in neighbours:
          if self.one == False:
              if isinstance(neighbor, GameLifeAgent):
                    #Si la baldosa esta sucia, hay carga y esta a 1 de distancia 
                    #entonces la limpia y se mueve hacia la celda donde esta la baldosa sucia
                  if neighbor.live != 3 and self.charge > 0 and (abs(self.pos[0] - neighbor.pos[0])) <= 1 and (abs(self.pos[1] - neighbor.pos[1])) <= 1:
                        #Se mueve al agente
                        self.model.grid.move_agent(self, neighbor.pos)
                        #Cuenta cantidad de movimientos hechos por la aspiradora
                        self.count += 1 
                        neighbor.live = 3
                        self.charge -= 20
                        #Variable que controla que nada mas se aspire una vez por turno
                        self.one == True
        
        # Si la aspiradora no encuentra basura en uno de los recuardos vecinos, 
        #selecciona uno al azar y se mueve hacia el
        if self.one == False and self.charge > 0:
            #Conseguir todas las celdas alrededor
            move = self.model.grid.get_neighborhood(
                      self.pos,
                      moore=True,
                      include_center=False)
            #Escoge una nueva posicion al azar
            newPos = self.random.choice(move)
            #Si la nueva posicion esta a una distancia de 1 de la posicion actual
            #entonces se mueve a esa celda y pierde algo de carga
            if (abs(self.pos[0] - newPos[0])) <= 1 and (abs(self.pos[1] - newPos[1])) <= 1 :
                  self.charge -= 5
                  self.model.grid.move_agent(self, newPos)
                  #Variable que cuenta movimientos de aspiradora
                  self.count += 1

        self.one = False
                          
      
    # Clase que define y posiciona las casillas limpias y sucias
class GameLifeAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.live = np.random.choice([0,3]) 
        self.next_sate = None

       
    # Clase que define El grid y las reglas del juego
class GameLifeModel(Model):
    def __init__(self, width, height):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        i = 0
        
        #Por cada celda se ponen las baldosas y las aspiradoras
        for (content, x, y) in self.grid.coord_iter():
            #La variable i representa la cantidad de aspiradoras que se quieren. 
            #Se pueden llegar a tener hasta 100 aspiradoras que es la cantidad de celdas
            if i < 12:
              b = GameLifeVacuum((x,y,i), self)
              #Se pone la aspiradora en la primera fila y en cualquier ubicacion de "y"
              self.grid.place_agent(b, (0, y))
              self.schedule.add(b)

            i=i+1

            #Cada celda tiene una baldosa que en si esta sucia o limpia
            a = GameLifeAgent((x, y), self)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)
            
        
        # Aquí definimos el colector de datos para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": self.get_grid}
        )
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def get_grid(self):
        # Generamos la grid para contener los valores
        grid = np.zeros((self.grid.width, self.grid.height))

        # Asignamos una celda a cada uno de los elementos de la grilla
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            #Se itera por cada celda del grid, se sacan los objetos y se pinta la celda
            for obj in cell_content:
              if isinstance(obj, GameLifeVacuum):
                grid[x][y] = 1
              elif isinstance(obj, GameLifeAgent):
                grid[x][y] = obj.live
        return grid
 

# Ejecución del modelo
A continuación corremos el modelo

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 10

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 100

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = GameLifeModel(GRID_SIZE, GRID_SIZE)
for i in range(NUM_GENERATIONS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

#Variables para sacar el porcentaje
clean = 0
total = 0
i = 0
#Se itera por todo el grid
for cell in model.grid.coord_iter():
  cell_content, x, y = cell
  for obj in cell_content:  
    if isinstance(obj, GameLifeVacuum):
        i += 1
        #Se imprime los movimientos de cada  aspiradora
        print('Movimientos Aspiradora {0}: {1}'.format(i, obj.count))
    if isinstance(obj, GameLifeAgent):
      #Se calcula total de baldosas
      total += 1
      if obj.live == 3:
          #Calcula baldosas limpias
          clean += 1

#Calcula porcentaje
print('Porcentaje:', (clean/total)*100)



Tiempo de ejecución: 0:00:00.048373
Movimientos Aspiradora 1: 73
Movimientos Aspiradora 2: 76
Movimientos Aspiradora 3: 82
Movimientos Aspiradora 4: 94
Movimientos Aspiradora 5: 9
Movimientos Aspiradora 6: 10
Movimientos Aspiradora 7: 44
Movimientos Aspiradora 8: 42
Movimientos Aspiradora 9: 95
Movimientos Aspiradora 10: 89
Movimientos Aspiradora 11: 41
Movimientos Aspiradora 12: 40
Porcentaje: 100.0


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])

#Los colores utilizados
cmap = matplotlib.cm.get_cmap('magma', 7)
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([0/256, 0/256, 0/256, 1])   
cmap[1] = np.array([0/256, 0/256, 0/256, 1])    
cmap[2] = np.array([0/256, 0/256, 139/256, 1])   #Azul
cmap[3] = np.array([0/256, 0/256, 0/256, 1])   


new_cmap = matplotlib.colors.ListedColormap(cmap)

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)



def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim